## Info before preprocesing

In [1]:
import pickle
import numpy as np
from Vectorisation import Vectorisation
from Config import Config
import masking

In [2]:
# I manually move the data (locally) inside this folder
with open("./ml4science_data.pkl", "rb") as fp:
    dl = pickle.load(fp)

In [3]:
print(dl.keys())
dl["sequences"][0].keys()

dict_keys(['sequences', 'index', 'available_demographics'])


dict_keys(['path', 'length', 'learner_id', 'raw_sequence', 'raw_begin', 'raw_end', 'last_timestamp', 'permutation', 'gender', 'year', 'language', 'begin', 'end', 'break_sequence', 'sequence', 'label', '1hot-sequence', 'nobreak', 'nobreak-1hot-sequence', 'stratifier_column'])

In [4]:
for i in range(6):
    print(
        "begin {} - end {}: {}".format(
            dl["sequences"][0]["begin"][i],
            dl["sequences"][0]["end"][i],
            dl["sequences"][0]["sequence"][i],
        )
    )

begin 0 - end 7.889: [0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3440621047581669]
begin 7.889 - end 83.03: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0]
begin 83.03 - end 86.28: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14174189890531658]
begin 86.28 - end 86.33: [0.0, 0.0, 0.0, 1.0, 0.006079027355622759, 0.0, 0.0, 0.0, 0.0, 0.0]
begin 87.278 - end 90.562: [0.0, 0.0, 0.0, 1.0, 0.3992705167173245, 0.0, 0.0, 0.0, 0.0, 0.0]
begin 90.562 - end 99.597: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3940424789567799]


In [5]:
student_seq = [len(dl["sequences"][i]["sequence"]) for i in range(len(dl["sequences"]))]

print(np.mean(student_seq), "+-", np.std(student_seq))
print(np.min(student_seq))
print(np.max(student_seq))

100.31496062992126 +- 102.75095652848267
13
819


In [6]:
config = Config()
vec = Vectorisation(config=config)

encoded = vec.encode_dict(dl)

## Testing for Vectorisation with dict
### Encoding

In [7]:
stud0 = dl["sequences"][0]
for i in range(20):
    print(
        f"begin {stud0['begin'][i]} - end {stud0['end'][i]}: \t {stud0['sequence'][i]} \t -> {encoded[0][i]}"
    )

begin 0 - end 7.889: 	 [0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3440621047581669] 	 -> 26
begin 7.889 - end 83.03: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0] 	 -> 2
begin 83.03 - end 86.28: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14174189890531658] 	 -> 26
begin 86.28 - end 86.33: 	 [0.0, 0.0, 0.0, 1.0, 0.006079027355622759, 0.0, 0.0, 0.0, 0.0, 0.0] 	 -> 21
begin 87.278 - end 90.562: 	 [0.0, 0.0, 0.0, 1.0, 0.3992705167173245, 0.0, 0.0, 0.0, 0.0, 0.0] 	 -> 21
begin 90.562 - end 99.597: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3940424789567799] 	 -> 26
begin 99.597 - end 100.32: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.1239499399965717, 0.0, 0.0, 0.0] 	 -> 23
begin 101.267 - end 107.071: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0772414527355239, 0.0] 	 -> 25
begin 108.989 - end 114.653: 	 [0.0, 0.0, 0.0, 1.0, 0.6886322188449855, 0.0, 0.0, 0.0, 0.0, 0.0] 	 -> 21
begin 115.424 - end 115.935: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.08760500600034259, 0.0,

In [8]:
print(encoded[0])

[26  2 26 21 21 26 23 25 21 23  2 21 26 21 21 26 21 25  3  8  3  3  3  3
  3  3  3  3  7  8  4  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
  9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


### Decoding

In [9]:
decoded = vec.decode_dict(encoded)

stud0 = dl["sequences"][0]
for i in range(20):
    print(
        f"begin {stud0['begin'][i]} - end {stud0['end'][i]}: \t {stud0['sequence'][i]} \t -> {encoded[0][i]} -> \t {decoded['sequences'][0]['sequence'][i]}"
    )

begin 0 - end 7.889: 	 [0, 0, 0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3440621047581669] 	 -> 26 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
begin 7.889 - end 83.03: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.9999999999999999, 0.0] 	 -> 2 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
begin 83.03 - end 86.28: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14174189890531658] 	 -> 26 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
begin 86.28 - end 86.33: 	 [0.0, 0.0, 0.0, 1.0, 0.006079027355622759, 0.0, 0.0, 0.0, 0.0, 0.0] 	 -> 21 -> 	 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
begin 87.278 - end 90.562: 	 [0.0, 0.0, 0.0, 1.0, 0.3992705167173245, 0.0, 0.0, 0.0, 0.0, 0.0] 	 -> 21 -> 	 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
begin 90.562 - end 99.597: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3940424789567799] 	 -> 26 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
begin 99.597 - end 100.32: 	 [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.1239499399965717, 0.0, 0.0, 0.0] 	 -> 23 -> 	 [0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
begin 101.267 - end 107.071: 	 [0.0, 0.0, 0.0

## Testing for Vectorisation without dict
### Encoding

In [10]:
stud1 = [
    [1, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
]

stud2 = [
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
]

stud3 = [[0, 0, 0, 1, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]]

studs = [stud1, stud2, stud3]
type(studs)

list

In [11]:
config2 = Config(MAX_LEN=6)
vec2 = Vectorisation(config=config2)

# without defining breaks
enc_no_sep = vec2.encode(studs)
print(enc_no_sep)

[[ 3  7  8 21 21 26]
 [25 25 26 21 21  0]
 [26 25  0  0  0  0]]


In [12]:
# with breaks
seps = np.array(
    [[0, 1, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0]], dtype=bool
)
enc_sep = vec2.encode(studs, seps)
print(enc_sep)

[[ 3  2  8 21 21 26]
 [ 2  2 26 21 21  0]
 [26  2  0  0  0  0]]


### Decoding

In [13]:
dec = vec2.decode(enc_sep)

for i in range(len(dec)):
    for j in range(len(dec[i])):
        print(f"{studs[i][j]} \t -> \t {enc_sep[i][j]} \t -> \t {dec[i][j]}")
    print("\n")

[1, 0, 0, 0, 1, 0, 0, 0, 0, 0] 	 -> 	 3 	 -> 	 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 1, 0] 	 -> 	 2 	 -> 	 [1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1] 	 -> 	 8 	 -> 	 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0] 	 -> 	 21 	 -> 	 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0] 	 -> 	 21 	 -> 	 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1] 	 -> 	 26 	 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]


[0, 0, 0, 1, 0, 0, 0, 0, 1, 0] 	 -> 	 2 	 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0] 	 -> 	 2 	 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 1, 0, 0, 0, 0, 0, 1] 	 -> 	 26 	 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0] 	 -> 	 21 	 -> 	 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0] 	 -> 	 21 	 -> 	 [0, 0, 0, 1, 1, 0, 0, 0, 0, 0]


[0, 0, 0, 1, 0, 0, 0, 0, 0, 1] 	 -> 	 26 	 -> 	 [0, 0, 0, 1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 1, 0, 0, 0, 0, 1, 0] 	 -> 	 2 	 -> 	 [0, 0,

# Masking Test

In [21]:
encoded_masked, labels, weights = masking.mask_input_and_labels(encoded, config.TOKEN_DICT)

In [22]:
print("Original: ", encoded[0])
print("Masked: ", encoded_masked[0])

Original:  [26  2 26 21 21 26 23 25 21 23  2 21 26 21 21 26 21 25  3  8  3  3  3  3
  3  3  3  3  7  8  4  2  8  3 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13 14 10 11
  9  9 13 14 13 14 11  9 10 14 13 10 11  9 14  9 14 15 20 15 15 17 20  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
Masked:  [26  2 26 21 21 26 23  8 21  1  2 21 26 21 21 26 21  1  3  8  3  3  3  1
  3  3  3  3  7  8  4  2  8  1 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11

In [16]:
percentage_ones = (
    np.count_nonzero(encoded_masked == 1, axis=1)
    / np.count_nonzero(encoded_masked, axis=1)
    * 100
)
print(percentage_ones)

print(
    "Total %:",
    percentage_ones.mean(),
    "+-",
    percentage_ones.std(),
    "\nExpected %:",
    0.15 * 0.8,
    "(0.15% * 0.8%)",
)
# Small % than expected cuz we don't mask SEP

[10.52631579  9.32642487 18.75       13.41463415 12.90322581  9.375
 11.76470588 17.33333333 21.05263158  2.63157895 15.68627451  4.76190476
  6.29370629 13.01369863 11.32075472 10.79545455 18.60465116 10.60606061
 16.66666667  6.25       12.73584906  7.14285714 15.78947368  9.47368421
  6.25       11.62790698  0.         13.36898396  8.10810811 10.11235955
 14.58333333  8.92857143 12.31527094 11.11111111 16.93989071 13.43283582
  5.55555556 12.61261261 12.06896552 11.96172249  9.80392157 13.88888889
 15.27777778 18.75       13.07692308 17.39130435 15.78947368  0.
  7.5        11.328125    8.984375    8.36653386  9.43396226  0.
  0.         14.          6.16438356 11.11111111  2.94117647 18.24324324
  6.52173913 13.59223301 14.58333333 20.68965517  7.31707317  9.4972067
  8.33333333 10.34482759 14.28571429 10.9375     10.52631579 13.15789474
 12.59259259 10.86956522 10.25641026 20.         14.15929204  3.44827586
 13.92405063  7.69230769 14.81481481 10.9375     10.25641026 15.27777778


In [28]:
# labels just correspond to the unmasked data
np.all(labels == encoded)

True

In [23]:
print(weights[0])  # has 1 where there is a 1 in the masked sequence (masked)
print(encoded_masked[0])

[0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[26  2 26 21 21 26 23  8 21  1  2 21 26 21 21 26 21  1  3  8  3  3  3  1
  3  3  3  3  7  8  4  2  8  1 16 20 21 26 21 23 21  3  8  7  8  3  5  8
  4  8  2  8  3  8 10  9 10  9  9 15 20 11  9 14 13 11  9 14 13  1  1 11
 